In [ ]:
import cv2
import time
import numpy as np
import pandas as pd
import mediapipe as mp
import plotly.express as px
import plotly.graph_objects as go

## Pose Detection + Gaze

In [ ]:
ANGLE_BGR_COLOR = (0,165,255)
POSE_BGR_COLOR = (255,0,0)
FPS_BGR_COLOR = (0,0,255)

In [ ]:
class poseDetector:
    def __init__(
        self,
        mode=False,
        complex=1,
        smooth_landmarks=True,
        segmentation=True,
        smooth_segmentation=True,
        detectionCon=0.5,
        trackCon=0.5,
    ):

        self.mode = mode
        self.complex = complex
        self.smooth_landmarks = smooth_landmarks
        self.segmentation = segmentation
        self.smooth_segmentation = smooth_segmentation
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpDraw = mp.solutions.drawing_utils
        self.mpDrawStyle = mp.solutions.drawing_styles
        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(
            self.mode,
            self.complex,
            self.smooth_landmarks,
            self.segmentation,
            self.smooth_segmentation,
            self.detectionCon,
            self.trackCon,
        )
        self.mp_drawing = mp.solutions.drawing_utils

    def findPose(self, img, draw = True):
        '''
        Pose estimation and drawing of the pose estimation to given frame.
        '''
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)    
        # self.plotly_fig(self.results.pose_landmarks)   
        # print(self.results.pose_landmarks)
          
        # Code to capture all body points and map it to a video
        if self.results.pose_landmarks:
            if draw:
                self.mp_drawing.draw_landmarks(
                    img,
                    self.results.pose_landmarks,
                    self.mpPose.POSE_CONNECTIONS,
                    self.mpDrawStyle.get_default_pose_landmarks_style())
                    # self.mpDraw.DrawingSpec(
                    #     color=(0, 0, 255), thickness=2, circle_radius=2
                    # ),
                    # self.mpDraw.DrawingSpec(
                    #     color=(0, 255, 0), thickness=2, circle_radius=2
                    # ),
        return img

    def findPosition(self, img, draw=False):
        '''
        Finds position of all the landmarks from previous pose estimation.
        '''
        self.lmList = []
        if self.results.pose_landmarks:
            # Save landmarks
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                # print(id, lm)
                cx, cy = int(lm.x * w), int(lm.y * h)
                x, y, z = lm.x, lm.y, lm.z
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (0, 255, 0), cv2.FILLED)
        return self.lmList

    def findAngle(self, img, p1, p2, p3, draw=True):
        # Get the landmarks
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        x3, y3 = self.lmList[p3][1:]

        # Calculate the Angle
        radians = np.arctan2(y3 - y2, x3 - x2) - np.arctan2(y1 - y2, x1 - x2)
        angle = radians * 180.0 / np.pi # np.abs(radians * 180.0 / np.pi)

        if angle > 180.0:
            angle = -(360 - angle)
        elif angle < -180:
            angle += 360

        # print(int(angle))

        # Draw
        if draw:
            # cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 3)
            # cv2.line(img, (x3, y3), (x2, y2), (255, 255, 255), 3)
            # cv2.circle(img, (x1, y1), 5, (0, 0, 255), cv2.FILLED)
            # cv2.circle(img, (x1, y1), 10, (0, 0, 255), 2)
            # cv2.circle(img, (x2, y2), 5, (0, 0, 255), cv2.FILLED)
            # cv2.circle(img, (x2, y2), 10, (0, 0, 255), 2)
            # cv2.circle(img, (x3, y3), 5, (0, 0, 255), cv2.FILLED)
            # cv2.circle(img, (x3, y3), 10, (0, 0, 255), 2)
            cv2.putText(
                img,
                str(int(angle)) + "",
                (x2 - 50, y2 + 50),
                cv2.FONT_HERSHEY_PLAIN,
                2,
                ANGLE_BGR_COLOR,# (255, 0, 0),
                2,
            )
        return angle

In [ ]:
OUTPUT_DIR = 'out'
VIDEO_OUT_PATH = 'out.mp4'
!mkdir out

mkdir: cannot create directory ‘out’: File exists


In [ ]:
size = (1920, 1080)
fourcc = cv2.VideoWriter_fourcc(*'MP4V') # cv2.VideoWriter_fourcc(*'DIVX') 
FPS = 25
out = cv2.VideoWriter(VIDEO_OUT_PATH,fourcc,FPS, size)

In [ ]:
GAZE_LINE_THICNKESS = 1
GAZE_LINE_COLOR = (0,0,255)

In [ ]:
def gaze(img, detector):
    left_eye_coordinates = detector.lmList[2][1:]
    right_eye_coordinates = detector.lmList[5][1:]
    
    mid_eye_point = (int((left_eye_coordinates[0] + right_eye_coordinates[0]) / 2), 
                     int((left_eye_coordinates[1] + right_eye_coordinates[1]) / 2))
    nose_point = detector.lmList[0][1:]

    theta = np.arctan2(mid_eye_point[1] - nose_point[1], mid_eye_point[0] - nose_point[0])
    endpt_x = int(mid_eye_point[0] - 1000*np.cos(theta))
    endpt_y = int(mid_eye_point[1] - 1000*np.sin(theta))

    # Mid eye point -> nose line
    return cv2.line(img, mid_eye_point, (endpt_x, endpt_y), GAZE_LINE_COLOR, GAZE_LINE_THICNKESS)

In [ ]:
def main():
    cap = cv2.VideoCapture('Hackathon_1st_Hitter.mp4')
    milliseconds = 1000
    start_time = int(input("Enter Start time (in seconds): "))
    end_time = int(input("Enter Length (in seconds): "))
    end_time = start_time + end_time
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time * milliseconds)
    # pTime = 0
    detector = poseDetector()
    cnt = 0
    while True and cap.get(cv2.CAP_PROP_POS_MSEC) <= end_time * milliseconds:
    # success = True
    # while success:
        success, img = cap.read()

        # Pose estimation + pose drawing
        img = detector.findPose(img, draw=True)
        lmList = detector.findPosition(img, draw=False)#, #draw=True)

        if len(lmList) != 0:
            detector.findAngle(img, 13, 11, 23)
            detector.findAngle(img, 24, 12, 14)
        
        # Gaze + drawing
        if detector.lmList:
            img = gaze(img, detector)

        # Save output image
        # cv2.imwrite("{}/image_{}.jpg".format(OUTPUT_DIR, cnt), img)
        
        # Output frame to the video
        out.write(img)

        cnt += 1
        # if cnt >= 3:
        #     break

    out.release()

In [ ]:
main()

In [ ]:
# x = [[0, 885, 191, 0.9997202157974243], [1, 879, 176, 0.9997184872627258], [2, 874, 175, 0.9997063279151917], [3, 870, 174, 0.9996131658554077], [4, 890, 177, 0.9997570514678955], [5, 893, 177, 0.9997616410255432], [6, 896, 176, 0.99972003698349], [7, 864, 181, 0.9994305968284607], [8, 900, 182, 0.9997619986534119], [9, 878, 201, 0.999589204788208], [10, 890, 202, 0.9996969103813171], [11, 801, 268, 0.9998000264167786], [12, 957, 260, 0.9999763369560242], [13, 767, 377, 0.9374921321868896], [14, 1021, 382, 0.9838160276412964], [15, 770, 421, 0.6886430978775024], [16, 1001, 464, 0.665144681930542], [17, 757, 431, 0.582179069519043], [18, 999, 494, 0.5551601648330688], [19, 769, 427, 0.572348952293396], [20, 993, 491, 0.5417897701263428], [21, 772, 420, 0.49530112743377686], [22, 982, 474, 0.48708102107048035], [23, 839, 483, 0.9997233748435974], [24, 908, 492, 0.9999595880508423], [25, 885, 607, 0.7366760969161987], [26, 886, 659, 0.9951562881469727], [27, 925, 731, 0.920708954334259], [28, 850, 846, 0.9939728379249573], [29, 935, 757, 0.9209998846054077], [30, 828, 879, 0.9771071076393127], [31, 966, 739, 0.8682690858840942], [32, 911, 864, 0.9716724753379822]]
# sorted(x, key=lambda x: x[3])[:10]

In [ ]:
import multiprocessing

multiprocessing.cpu_count()

# TODO - multithread processing for Mediapipe Pose and batch processing for YOLOv5

4

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=380d27e3-c5ba-4379-bcc3-a75537d76331' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>